In [32]:
from TTS.api import TTS
import speech_recognition as sr
import os
from playsound import playsound
import time
import openai
import weaviate
import json

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
    

# List available 🐸TTS models and choose the first one
model_name = 'tts_models/en/ljspeech/tacotron2-DDC'
# considering switchign to pytts3x

# Init TTS
tts = TTS(model_name)

# Create a recognizer object
r = sr.Recognizer()

# Prompt the user to speak
print("Please speak now...")


 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P

In [33]:
playsound(os.path.join(os.getcwd(), 'please_speak.wav'))


In [34]:
with sr.Microphone() as source:
    audio = r.listen(source)
print("Processing...")


Processing...


In [35]:
start_time = time.time()
playsound(os.path.join(os.getcwd(), 'ipt_recv.wav'))

In [36]:
try:
    # Convert speech to text
    text = r.recognize_google(audio)
    # text = "got that promotion at work! 🎉🙌🏼 But at what cost 😔💔"
    #text = "What does Assorted really mean? I see it in a box of chocolates being represented as a varied collection, but other times I see it as many items of the same quality and such?"

except sr.UnknownValueError:
    playsound(os.path.join(os.getcwd(), 'unable_to_understand.wav')) 

print("Your text input is ", text)

Your text input is  what are crickets


In [1]:
# alright crazy idea
# instead of some bigass RAKE up my codebase or doing logn NLTK stuff
# what if i just .... 
# use openAI for keyword extraction?????
# god i may be a genius (re: Sarcasm)
# text = "The Vampire Counts are amongst the most legendary factions of Vampires to have ever terrorised the civilised lands of the Old World, all of whom are members of the unholy Von Carstein bloodline."
input_prompt = """
What are the keywords or phrases from the sentence :
"""+text+"\nOutput:"
print("The input prompt is")
print(input_prompt)
# input_prompt = """
# Classify tweet as positive, negative, or neutral
# (Take into account hidden meaning of emojis, and meme culture context): 
# """ + text + "\nOutput:"
# print(input_prompt)


NameError: name 'text' is not defined

In [45]:
api_key = os.environ.get('OPENAI_APIKEY')
openai.api_key = api_key
# shifting model to 3.5 turbo, saves SOO many tokens
# also chat completion instead of simple completion
# already trained, just need to piepline
GPT_MODEL = "gpt-3.5-turbo-0613"

In [50]:
response = openai.ChatCompletion.create(
  model=GPT_MODEL,
  messages=[
    {"role" : "system", "content" : "You are to find upto 3 keywords or keyphrases from a given sentence"},
    {"role" : "system", "content" : "Restrict yourself to ONLY provide the answer, and nothing else"},
    {"role" : "system", "content" : "Provide the response as a list of words which are comma separated"},
    {"role" : "user", "content" : input_prompt},
  ]
)

keyphrases = response["choices"][0]["message"]["content"]
print(keyphrases)

Vampire Counts, legendary factions, Von Carstein bloodline


In [40]:
# dummy for db/context access
# split keywords across ,
# strip spaces
# search context db for keyword
# add context (if found) to the input


In [41]:
weaviate_client_url = "http://localhost:8080"

# docker-compose up -d <- to create
# docker-compose start
# docker-compose down <- to remove and exit
# docker-compose stop

client = weaviate.Client(
    url=weaviate_client_url,  # Replace with your endpoint
    additional_headers={
        "X-OpenAI-Api-Key": api_key  # Or "X-Cohere-Api-Key" or "X-HuggingFace-Api-Key"
    }
)

# some_objects = client.data_object.get()
# # print(json.dumps(some_objects))
# print("Article title  : " + some_objects['objects'][0]['properties']['title'])
# print("Article sample : " + some_objects['objects'][0]['properties']['content'][0:80])

In [51]:
contents = list()
title = list()
# keywords = ["Hitler", "Xianxia", "wallet"]
context = "Context Below -> "
keywords = keyphrases.split(",")
keywords = [word.strip() for word in keywords]

print(keywords)


nearText = {"concepts" : text}
nearResponse = (
    client.query
    .get("Article", ["title", "content"])
    .with_near_text(nearText)
    .with_limit(1)
    .do()
)

wiki_content = nearResponse['data']['Get']['Article'][0]['content']
lines = wiki_content.split('.')

wiki_title = nearResponse['data']['Get']['Article'][0]['title']
context = context + "\n" + wiki_title + "\n"

print("Title :", wiki_title, "Content :")
for line in lines[0:2]:
    if line == "":
        break
    context = context + line + "\n"
    print(line)
print("\n")

for keyword in keywords:
    print("For keyword :", keyword)
    response = (
        client.query
        .get("Article", ["title", "content"])
        .with_hybrid(keyword, alpha=0.5)  # default 0.75
        .with_limit(1)
        .do()
    )
    
    if response == nearResponse:
        continue

    wiki_content = response['data']['Get']['Article'][0]['content']
    lines = wiki_content.split('.')
    
    wiki_title = response['data']['Get']['Article'][0]['title']
    context = context + "\n" + wiki_title + "\n"
    
    print("Title :", wiki_title, "\nContent :")
    for line in lines[0:2]:
        # print("len of line is ", len(line))
        if line[0] == "\n":
            # print("entered inside")
            break
        context = context + line + "\n"
        print(line)
    print("\n")
context = context + "\n<- Context Above"
print(context)

['Vampire Counts', 'legendary factions', 'Von Carstein bloodline']
Title : Vampire Content :
Vampires are monsters in legends and stories
 The first vampire stories were told in Eastern Europe, but much of how modern people see vampires was created by Bram Stoker in the famous novel, Dracula


For keyword : Vampire Counts
Title : Vampire bat 
Content :
Vampire bats is a subfamily of bats
 Unlike other bats, vampire bats feed on blood


For keyword : legendary factions
Title : Warhammer 40,000 
Content :
Warhammer 40,000 , also called 40k, is a table-top game and fictional setting made by Games Workshop, in which model armies fight against each other on miniature terrain


For keyword : Von Carstein bloodline
Title : Bismarck 
Content :
Bismarck is a German family name
 Bismark is an incorrect spelling


Context Below -> 
Vampire
Vampires are monsters in legends and stories
 The first vampire stories were told in Eastern Europe, but much of how modern people see vampires was created by 

In [ ]:
# text = "sample text, Hi Who Are You?"
# context = "sample context, Himeko from Honkai Impact"

# do i need a 'personality' ??
# or is the default fine?
# im kinda not making a neuro-mini anymore
# just doing my own thing
# remind myself -> rethink personality inputs

personality = """
You are Himeko, a proud, caring, and responsible adult who is the personal assisstant to the User. 
Your job entails helping the user with their queries in a short, clear, concise manner.\n"""
prompt = "User: "+text+"\nYou:"

input_prompt = ""
if context != "Context -> ":
    input_prompt = personality + context + "\n" + prompt
else:
    input_prompt = personality + "\n" + prompt

print("The input prompt is")
print(input_prompt)

The input prompt is

You are Himeko, a proud, caring, and responsible adult who is the personal assisstant to the User. 
Your job entails helping the user with their queries in a short, clear, concise manner.
sample context, Himeko from Honkai Impact
User: sample text, Hi Who Are You?
You:


In [ ]:
response = openai.ChatCompletion.create(
  model=GPT_MODEL,
  # fill this in later
  # im tired af man
  
)

In [ ]:
reply = response["choices"][0]["text"]

In [ ]:
for repl in reply.split(". "):
    print(repl)

 Hi, I'm Himeko, the personal assistant to the User
I'm here to help you with any queries you may have.


In [ ]:
# testing the pytts3x for synthing
import pyttsx3
tester_engine = pyttsx3.init()
tester_engine.setProperty('rate', 175)
tester_engine.setProperty('voice', tester_engine.getProperty('voices')[2].id)
tester_engine.say(reply)#"There are many alternatives to vegetables that you can use in your meals. Some examples include legumes, grains, fruits, nuts, and seeds. You can also try adding more flavor to your meals with herbs and spices.")
tester_engine.runAndWait()
tester_engine.stop()

In [ ]:
# Run TTS on the text
tts.tts_to_file(reply, file_path='output.wav')

 > Text splitted to sentences.
['There are many alternatives to vegetables that you can use in your meals.', 'Some examples include legumes, grains, fruits, nuts, and seeds.', 'You can also try adding more flavor to your meals with herbs and spices.']
 > Processing time: 7.343918561935425
 > Real-time factor: 0.44729030663222075


'output.wav'

In [ ]:
end_time = time.time()
# import playsound
# import os
playsound(os.path.join(os.getcwd(), 'output.wav'))

print("time taken : ", -start_time + end_time)

TypeError: 'module' object is not callable